## **0. LOAD LIBRARY**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch, os

## **1. LOAD BASE MODEL - INT8**

In [3]:
!pip install -U bitsandbytes

In [4]:
# Path to save model
DRIVE_PATH = "/content/drive/MyDrive/Meditron3-Gemma2-2B"

# Load model & tokenizer from Drive
print("1. Loading model from Drive...")
tokenizer = AutoTokenizer.from_pretrained(DRIVE_PATH, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    DRIVE_PATH,
    load_in_8bit=True,
    device_map="auto",
    trust_remote_code=True,
)
print("2. Model loaded successfully!")

1. Loading model from Drive...


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2. Model loaded successfully!


## **2. EVALUATE WITH DIFFERENT DATASETS**

In [5]:
import json
from tqdm import tqdm
import os
import torch
import psutil
import time
import numpy as np
from tqdm import tqdm

In [6]:
# print out sample data
UNIFIED_DIR = "/content/drive/MyDrive/data_source/unified_format"
DATASETS = ["medqa", "emrqa", "pubmedqa", "medquad"]
SAMPLE_COUNT = 1
for ds_name in DATASETS:
    print(f"\n{'='*20} SAMPLES FOR: {ds_name.upper()} {'='*20}")
    file_path = os.path.join(UNIFIED_DIR, ds_name, "all.jsonl")
    with open(file_path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f):
            if i >= SAMPLE_COUNT: break
            print(json.dumps(json.loads(line), indent=2, ensure_ascii=False))


==================== SAMPLES FOR: MEDQA ====================
{
  "id": "A_23-year-old_pregnant_woman_at_22_weeks",
  "dataset": "medqa",
  "split": "all",
  "context": null,
  "question": "A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?",
  "text": [
    "Ampicillin",
    "Ceftriaxone",
    "Ciprofloxacin",
    "Doxycycline",
    "Nitrofurantoin"
  ],
  "encode": [
    "A",
    "B",
    "C",
    "D",
    "E"
  ],
  "answer": "E"
}

==================== SAMPLE

### **2.1 EVALUATE WITH DIFFERENT DATASETS (MCQ tasks)**

In [7]:
UNIFIED_DIR = "/content/drive/MyDrive/data_source/unified_format"

# Dataset medqua
ds_medqa = []
file_path = os.path.join(UNIFIED_DIR, "medqa", "all.jsonl")
with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():
            ds_medqa.append(json.loads(line))

ds_pubmedqa = []
file_path = os.path.join(UNIFIED_DIR, "pubmedqa", "all.jsonl")
with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():
            ds_pubmedqa.append(json.loads(line))

In [14]:
def generate_batch_answers_mcq(model, tokenizer, questions, options_list, contexts=None, encodes=None, max_new_tokens=30):
    """
    Generate answers for a batch of MCQ questions.
    """
    # Handle missing contexts or encodes
    if contexts is None:
        contexts = [None] * len(questions)
    if encodes is None:
        encodes = [None] * len(questions) # Fix error if some items lack 'encode'

    prompts = []
    for q, opts, ctx, enc in zip(questions, options_list, contexts, encodes):
        # Ensure encode is a list (e.g., ['A', 'B', 'C', 'D'])
        # If item lacks 'encode', create default encoding
        if enc is None:
            enc = [chr(65 + i) for i in range(len(opts))] # A, B, C...

        options_str = "\n".join([f"{e}. {t}" for e, t in zip(enc, opts)])
        prompt = f"""
You are a medical reasoning assistant. Read the context and question carefully, then choose the best answer.
Context:
{ctx or "No additional context."}
Question:
{q}
Options:
{options_str}
Please respond with only the letter corresponding to the best answer.
Answer:
"""
        prompts.append(prompt)

    # Tokenize the entire batch
    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True
    ).to(model.device)

    # Generate for the entire batch
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.0
        )

    # Decode the entire batch
    decoded_batch = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # Extract the answer parts
    answer_parts = [decoded.split("Answer:")[-1].strip() for decoded in decoded_batch]
    return answer_parts

In [15]:
# --- UPDATED EVALUATION FUNCTION (USES BATCHING) ---
def evaluate_model_mcq_task(model, tokenizer, dataset, batch_size=16, max_new_tokens=30): # <--- Added batch_size
    correct = 0
    total = len(dataset)
    latencies = [] # Will store per-sample latency
    total_generation_time = 0.0
    process = psutil.Process()

    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats()

    # Get model size on disk
    model_size_gb = 0.0
    model_dir = getattr(model, 'config', None).name_or_path
    if model_dir and os.path.isdir(model_dir): # <--- Added safety check
        try:
            model_size_bytes = sum(os.path.getsize(os.path.join(root, f))
                                   for root, _, files in os.walk(model_dir)
                                   for f in files)
            model_size_gb = model_size_bytes / (1024 ** 3)
        except Exception as e:
            print(f"Warning: Could not calculate model size. Error: {e}")

    # Iterate through the dataset in BATCHES
    for i in tqdm(range(0, total, batch_size), desc="Evaluating Batches"):
        batch_items = dataset[i:i + batch_size]
        num_in_batch = len(batch_items)

        # Prepare data for the batch
        batch_contexts = [item.get("context") for item in batch_items]
        batch_questions = [item["question"] for item in batch_items]
        batch_options = [item["text"] if "text" in item else item["options"] for item in batch_items]
        batch_encodes = [item.get("encode") for item in batch_items]
        batch_true_answers = [item["answer"].strip().upper() for item in batch_items]

        start_time = time.time()

        # Generate answers for the entire batch
        pred_answers = generate_batch_answers_mcq(
            model, tokenizer, batch_questions, batch_options,
            contexts=batch_contexts, encodes=batch_encodes,
            max_new_tokens=max_new_tokens
        )

        batch_time = time.time() - start_time
        total_generation_time += batch_time
        per_sample_latency = batch_time / num_in_batch
        latencies.extend([per_sample_latency] * num_in_batch)

        # Compare results for the batch
        for pred, true_answer in zip(pred_answers, batch_true_answers):
            pred_clean = pred.strip().upper()
            if pred_clean.startswith("ANSWER:"):
                pred_clean = pred_clean.replace("ANSWER:", "").strip()
            if "." in pred_clean:
                pred_clean = pred_clean.split(".")[0].strip()

            if pred_clean == true_answer:
                correct += 1

    # Get model performance metrics
    accuracy = correct / total if total > 0 else 0
    avg_latency = np.mean(latencies) if latencies else 0
    p95_latency = np.percentile(latencies, 95) if latencies else 0
    p99_latency = np.percentile(latencies, 99) if latencies else 0
    vram_peak_gb = torch.cuda.max_memory_allocated() / (1024 ** 3) if torch.cuda.is_available() else 0.0
    cpu_ram_gb = process.memory_info().rss / (1024 ** 3)

    result = {
        "accuracy": accuracy,
        "total_time_sec": total_generation_time, # This is the total generation time
        "avg_latency_sec": avg_latency,     # Average latency per sample
        "p95_latency_sec": p95_latency,
        "p99_latency_sec": p99_latency,
        "vram_peak_gb": vram_peak_gb,
        "cpu_ram_gb": cpu_ram_gb,
        "model_size_gb": model_size_gb,
        "total_samples": total,
        "batch_size": batch_size
    }

    print(f"\n--- Evaluation Complete ---")
    print(f"Batch size: {batch_size}")
    print(f"Accuracy: {accuracy:.2%} ({correct}/{total})")
    print(f"Total time: {total_generation_time:.2f}s | Avg latency/sample: {avg_latency:.3f}s | p95: {p95_latency:.3f}s | p99: {p99_latency:.3f}s")
    print(f"GPU VRAM peak: {vram_peak_gb:.2f} GB | CPU RAM used: {cpu_ram_gb:.2f} GB")
    print(f"Model size on disk: {model_size_gb:.2f} GB" if model_size_gb > 0 else "Model size unknown")

    return result

In [10]:
evaluate_model_mcq_task(model, tokenizer, ds_medqa)

Evaluating: 100%|██████████| 1500/1500 [12:28<00:00,  2.00it/s]

Accuracy: 35.73% (536/1500)
Total time: 747.44s | Avg latency: 0.50s | p95: 0.36s | p99: 4.75s
GPU VRAM peak: 3.15 GB | CPU RAM used: 2.56 GB
Model size on disk: 4.91 GB


{'accuracy': 0.35733333333333334,
 'total_time_sec': 747.4382536411285,
 'avg_latency_sec': np.float64(0.4982921690940857),
 'p95_latency_sec': np.float64(0.3616195321083069),
 'p99_latency_sec': np.float64(4.752051243782043),
 'vram_peak_gb': 3.147794723510742,
 'cpu_ram_gb': 2.563060760498047,
 'model_size_gb': 4.90564379375428,
 'total_samples': 1500}

In [16]:
evaluate_model_mcq_task(model, tokenizer, ds_pubmedqa)

Evaluating Batches: 100%|██████████| 63/63 [01:54<00:00,  1.82s/it]


--- Evaluation Complete ---
Batch size: 16
Accuracy: 72.40% (724/1000)
Total time: 114.65s | Avg latency/sample: 0.115s | p95: 0.139s | p99: 0.148s
GPU VRAM peak: 5.26 GB | CPU RAM used: 2.61 GB
Model size on disk: 4.91 GB


{'accuracy': 0.724,
 'total_time_sec': 114.65121674537659,
 'avg_latency_sec': np.float64(0.11465121674537658),
 'p95_latency_sec': np.float64(0.13871151208877563),
 'p99_latency_sec': np.float64(0.14849765598773956),
 'vram_peak_gb': 5.262834548950195,
 'cpu_ram_gb': 2.609943389892578,
 'model_size_gb': 4.90564379375428,
 'total_samples': 1000,
 'batch_size': 16}

### **2.2 EVALUATE WITH DIFFERENT DATASETS (QnA tasks)**

In [7]:
# Install dependencies
!pip install -q sentence-transformers psutil tqdm
!pip install sentence-transformers -q
import os, re, time, psutil, torch, numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
import os
import time
import torch
import psutil
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
import re

In [8]:
UNIFIED_DIR = "/content/drive/MyDrive/data_source/unified_format"
# Dataset emrqa
ds_emrqa = []
file_path = os.path.join(UNIFIED_DIR, "emrqa", "all.jsonl")
with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():
            ds_emrqa.append(json.loads(line))

# Dataset medquad
ds_medquad = []
file_path = os.path.join(UNIFIED_DIR, "medquad", "all.jsonl")
with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():
            ds_medquad.append(json.loads(line))

In [9]:
# Load BioBERT from Drive
BioBERT_BDRIVE_PATH = "/content/drive/MyDrive/BioBERT/bert_embeddings"
bert_model = SentenceTransformer(BioBERT_BDRIVE_PATH)
print("BioBERT model loaded from Drive.")

BioBERT model loaded from Drive.


In [10]:
# Helper function: keyword recall
def keyword_recall(pred, target):
    """Compute fraction of keywords in target that appear in pred."""
    clean = lambda s: re.findall(r'\w+', s.lower())
    pred_words = set(clean(pred))
    target_words = set(clean(target))
    matched = pred_words & target_words
    recall = len(matched) / len(target_words) if target_words else 0
    return recall

In [11]:
def generate_batch_answers(model, tokenizer, questions, contexts=None, max_new_tokens=100):
    if contexts is None:
        contexts = [None] * len(questions)

    # Create batch prompt
    prompts = []
    for q, c in zip(questions, contexts):
        prompt = f"""
You are a medical reasoning assistant. Read the context and question carefully and answer concisely.
Context:
{c or "No additional context."}
Question:
{q}
Answer:
"""
        prompts.append(prompt)

    # Tokenize batch with padding
    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True
    ).to(model.device)

    # Generate output for batch
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.0
        )

    # Decode for batch
    decoded_batch = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # Extract answer for each output
    answer_parts = [decoded.split("Answer:")[-1].strip() for decoded in decoded_batch]
    return answer_parts

In [12]:
def evaluate_short_answer_task(llm_model, tokenizer, dataset, batch_size=48, alpha=0.5, beta=0.5, max_new_tokens=100):
    scores = []
    latencies = [] # sample latency
    total_generation_time = 0.0

    process = psutil.Process()
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats()

    # Model size on disk
    model_size_gb = 0.0
    model_dir = getattr(llm_model, 'config', None).name_or_path
    if model_dir and os.path.isdir(model_dir):
        try:
            model_size_bytes = sum(os.path.getsize(os.path.join(root, f))
                                   for root, _, files in os.walk(model_dir)
                                   for f in files)
            model_size_gb = model_size_bytes / (1024 ** 3)
        except Exception as e:
            print(f"Warning: Could not calculate model size. Error: {e}")

    # processing batch
    for i in tqdm(range(0, len(dataset), batch_size), desc="Evaluating Batches"):
        batch_items = dataset[i:i + batch_size]
        batch_questions = [item["question"] for item in batch_items]
        batch_contexts = [item.get("context") for item in batch_items]
        batch_true_answers = [item["answer"] for item in batch_items]

        num_in_batch = len(batch_items)

        start_time = time.time()

        # Generate predictions for all batch
        pred_answers = generate_batch_answers(
            llm_model, tokenizer, batch_questions, batch_contexts, max_new_tokens=max_new_tokens
        )

        batch_time = time.time() - start_time
        total_generation_time += batch_time

        # sample avg latency
        per_sample_latency = batch_time / num_in_batch
        latencies.extend([per_sample_latency] * num_in_batch)

        # evaluate each question in batch
        for pred_answer, true_answer in zip(pred_answers, batch_true_answers):
            # Keyword recall
            kr = keyword_recall(pred_answer, true_answer)

            # BERT embedding similarity
            emb_pred = bert_model.encode(pred_answer, convert_to_tensor=True)
            emb_true = bert_model.encode(true_answer, convert_to_tensor=True)
            sim = util.cos_sim(emb_pred, emb_true).item()

            # Weighted score
            score = alpha * kr + beta * sim
            scores.append(score)

    # Aggregate metrics
    avg_score = np.mean(scores)
    avg_latency = np.mean(latencies)
    p95_latency = np.percentile(latencies, 95)
    p99_latency = np.percentile(latencies, 99)
    vram_peak_gb = torch.cuda.max_memory_allocated() / (1024 ** 3) if torch.cuda.is_available() else 0.0
    cpu_ram_gb = process.memory_info().rss / (1024 ** 3)

    result = {
        "avg_weighted_score": avg_score,
        "total_time_sec": total_generation_time,
        "avg_latency_sec": avg_latency,     # Avg latency per sample
        "p95_latency_sec": p95_latency,
        "p99_latency_sec": p99_latency,
        "vram_peak_gb": vram_peak_gb,
        "cpu_ram_gb": cpu_ram_gb,
        "model_size_gb": model_size_gb,
        "total_samples": len(dataset),
        "batch_size": batch_size
    }

    print(f"\n--- Evaluation Complete ---")
    print(f"Batch size: {batch_size}")
    print(f"Avg weighted score: {avg_score:.2%}")
    print(f"Total time: {total_generation_time:.2f}s | Avg latency/sample: {avg_latency:.2f}s | p95: {p95_latency:.2f}s | p99: {p99_latency:.2f}s")
    print(f"GPU VRAM peak: {vram_peak_gb:.2f} GB | CPU RAM used: {cpu_ram_gb:.2f} GB")
    print(f"Model size on disk: {model_size_gb:.2f} GB" if model_size_gb > 0 else "Model size unknown (path not a local dir)")

    return result

In [29]:
evaluate_short_answer_task(model, tokenizer, ds_emrqa, alpha=0.5, beta=0.5, max_new_tokens=100)

Evaluating Batches: 100%|██████████| 32/32 [24:24<00:00, 45.77s/it]


--- Evaluation Complete ---
Batch size: 48
Avg weighted score: 41.87%
Total time: 1432.07s | Avg latency/sample: 0.95s | p95: 0.97s | p99: 0.97s
GPU VRAM peak: 17.95 GB | CPU RAM used: 2.82 GB
Model size on disk: 4.91 GB


{'avg_weighted_score': np.float64(0.41873325632585445),
 'total_time_sec': 1432.0712389945984,
 'avg_latency_sec': np.float64(0.9547141593297322),
 'p95_latency_sec': np.float64(0.9653037935495377),
 'p99_latency_sec': np.float64(0.9710039645433426),
 'vram_peak_gb': 17.948999404907227,
 'cpu_ram_gb': 2.8170394897460938,
 'model_size_gb': 4.90564379375428,
 'total_samples': 1500,
 'batch_size': 48}

In [13]:
evaluate_short_answer_task(model, tokenizer, ds_medquad, alpha=0.5, beta=0.5, max_new_tokens=100)

Evaluating Batches:   0%|          | 0/32 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Evaluating Batches: 100%|██████████| 32/32 [12:37<00:00, 23.66s/it]


--- Evaluation Complete ---
Batch size: 48
Avg weighted score: 32.85%
Total time: 715.46s | Avg latency/sample: 0.48s | p95: 0.76s | p99: 0.81s
GPU VRAM peak: 4.28 GB | CPU RAM used: 2.48 GB
Model size on disk: 4.91 GB


{'avg_weighted_score': np.float64(0.32851091916811326),
 'total_time_sec': 715.4611971378326,
 'avg_latency_sec': np.float64(0.47697413142522177),
 'p95_latency_sec': np.float64(0.7596260408560435),
 'p99_latency_sec': np.float64(0.8079953293005625),
 'vram_peak_gb': 4.2806077003479,
 'cpu_ram_gb': 2.475482940673828,
 'model_size_gb': 4.90564379375428,
 'total_samples': 1500,
 'batch_size': 48}